In [1]:
from pdfminer.high_level import extract_text

with open('/home/azashiro/Desktop/beetroot_task/task.pdf','rb') as file:
    theses = extract_text(file)
    t = " ".join(theses.split())
    plain_text = "".join(t)
    #print(plain_text)

In [213]:
import re

text = re.sub(r'Poster abstracts\s\d', "", plain_text)
text = re.sub(r'Poster abstractsActa Derm Venereol 2018\s\d+', "", text)
text = re.sub(r'www.medicaljournals.se/acta5th World Psoriasis & Psoriatic Arthritis Conference 2018', '', text)
text = re.sub(r'\w-\s', '', text)
text = re.sub(r'\xad', '', text)
#print(text)

In [3]:
pattern = re.compile(r"(P\d{3})")
text_list = []
text_list = re.split(pattern, text)
text_list.remove(text_list[0]) # remove page number
#print(text_list)

In [4]:
def convert_text_to_dictionary(text):
    it = iter(text)
    create_dict = dict(zip(it, it))
    return create_dict

text_dict = convert_text_to_dictionary(text_list)
#print(text_dict)

PARSE SINGLE ABSTRACT WITH NON-INDEXED AUTHORS LIST

In [189]:
get_abstract = text_dict['P111']
#get_abstract

In [190]:
abstract_info = re.findall(r"\w+[A-Z\w+]\w+.*(?=Introduction.*)", get_abstract)
abstract_head = str(abstract_info[0])
abstract_head

'SKIN LESION SEVERITY IN EARLY AXIAL AND PERIPHERAL PSORIATIC ARTHRITIS PATIENTS Elena Gubar, Elena Loginova, Svetlana Glukhova, Tatiana Kortaeva Nasonova Research Institute of Rheumatology, Moscow '

In [191]:
authors_info = re.findall(r"\w+[^A-Z\d)\W]\s\w.*(?=Introduction.*)", get_abstract)
authors = str(authors_info[0])
authors

'Elena Gubar, Elena Loginova, Svetlana Glukhova, Tatiana Kortaeva Nasonova Research Institute of Rheumatology, Moscow '

In [193]:
author_name = re.findall(r"\w.+(?=Nas)", authors)
author_name
author_list = [x for x in author_name[0].split(',')]
author_list

['Elena Gubar', ' Elena Loginova', ' Svetlana Glukhova', ' Tatiana Kortaeva ']

In [195]:
author_location = re.findall(r"Nas[^*]+", authors)
author_location

['Nasonova Research Institute of Rheumatology, Moscow ']

In [196]:
pattern = re.compile(r"\w+[^A-Z\d\W]\s\w.*")
abstract_title = [re.sub(pattern, "", i) for i in abstract_info]
abstract_title

['SKIN LESION SEVERITY IN EARLY AXIAL AND PERIPHERAL PSORIATIC ARTHRITIS PATIENTS ']

In [197]:
abstract = re.findall(r"(Introduction.*)", get_abstract)
abstract_text = abstract[0]
#abstract_text

In [22]:
import pandas as pd

df = pd.DataFrame({"About the person": 'Name (incl. titles if any mentioned)',
                   "Unnamed: 1": 'Affiliation(s) Name(s)',
                   "Unnamed: 2": "Person's Location",
                   "About the session/topic": "Session Name",
                   "Unnamed: 4": 'Topic Title',
                   "Unnamed: 5": 'Presentation Abstract'}, index=[0])

In [207]:
df1 = pd.DataFrame({"About the person": author_list[2],
                   "Unnamed: 1": author_location,
                   "Unnamed: 2": "",
                   "About the session/topic": "P106",
                   "Unnamed: 4": abstract_title,
                   "Unnamed: 5": abstract_text})
df1

,About the person,Unnamed: 1,Unnamed: 2,About the session/topic,Unnamed: 4,Unnamed: 5
0,Svetlana Glukhova,"Nasonova Research Institute of Rheumatology, M...",,P106,SKIN LESION SEVERITY IN EARLY AXIAL AND PERIPH...,Introduction: Comparative analysis of skin les...


PARSE SINGLE ABSTRACT WITH INDEXED AUTHORS LIST

In [209]:
get_abstract = text_dict['P112']
#get_abstract

In [126]:
abstract_info = re.findall(r"\w+[A-Z\w+]\w+.*(?=Background.*)", get_abstract)
abstract_head = str(abstract_info[0])
abstract_head

'GASTROINTESTINAL SYMPTOMS ARE COMMON IN U.S. PATIENTS WITH MODERATESEVERE PSORIASIS Steven Feldman1, Steve Fakharzadeh2, Jill Abell2, Timothy Hoops2, Bhaskar Srivastava2, Erik Muser2, Danielle Dungee2, Sean Quinn2, Megan Leone Perkins3, Michael Kappelman4 1Wake Forest School of Medicine, Winston-Salem, NC, 2Janssen Scientific Affairs, LLC, Horsham PA, 3HealthiVibe, LLC, Arlington, VA, 4University of North Carolina, Chapel Hill, NC, USA '

In [128]:
authors_info = re.findall(r"\w+[^A-Z\d)\W]\s\w.*(?=Background.*)", get_abstract)
authors = str(authors_info[0])
authors

'Steven Feldman1, Steve Fakharzadeh2, Jill Abell2, Timothy Hoops2, Bhaskar Srivastava2, Erik Muser2, Danielle Dungee2, Sean Quinn2, Megan Leone Perkins3, Michael Kappelman4 1Wake Forest School of Medicine, Winston-Salem, NC, 2Janssen Scientific Affairs, LLC, Horsham PA, 3HealthiVibe, LLC, Arlington, VA, 4University of North Carolina, Chapel Hill, NC, USA '

In [129]:
author_name = re.findall(r"(\w+.\s[a-zA-z\s-]+\d)", authors)
author_name

['Steven Feldman1',
 'Steve Fakharzadeh2',
 'Jill Abell2',
 'Timothy Hoops2',
 'Bhaskar Srivastava2',
 'Erik Muser2',
 'Danielle Dungee2',
 'Sean Quinn2',
 'Megan Leone Perkins3',
 'Michael Kappelman4']

In [130]:
author_location = re.findall(r"(\d\w+\W[a-zA-z-'\s&,\s.,(A-Z)]+)", authors)
author_location

['1Wake Forest School of Medicine, Winston-Salem, NC, ',
 '2Janssen Scientific Affairs, LLC, Horsham PA, ',
 '3HealthiVibe, LLC, Arlington, VA, ',
 '4University of North Carolina, Chapel Hill, NC, USA ']

In [103]:
import string
from collections import namedtuple

DigitGroup = namedtuple('DigitGroup', ['keys', 'values'])

def combine(all_keys, all_values):
    by_digit = {}

    for word in all_keys:
        for char in word:
            if char in string.digits:
                group = by_digit.get(char)
                if not group:
                    group = DigitGroup(word, [])
                by_digit[char] = group
                break

    for word in all_values:
        for char in word:
            if char in string.digits:
                group = by_digit[char]
                group.values.append(word)
                break
    
    return dict(by_digit.values())

In [131]:
combined_dict = combine(author_location, author_name)
combined_dict

{'1Wake Forest School of Medicine, Winston-Salem, NC, ': ['Steven Feldman1'],
 '2Janssen Scientific Affairs, LLC, Horsham PA, ': ['Steve Fakharzadeh2',
  'Jill Abell2',
  'Timothy Hoops2',
  'Bhaskar Srivastava2',
  'Erik Muser2',
  'Danielle Dungee2',
  'Sean Quinn2'],
 '3HealthiVibe, LLC, Arlington, VA, ': ['Megan Leone Perkins3'],
 '4University of North Carolina, Chapel Hill, NC, USA ': ['Michael Kappelman4']}

In [132]:
list_of_dictionaries = [{k: v} for k, v in combined_dict.items()]
list_of_dictionaries

[{'1Wake Forest School of Medicine, Winston-Salem, NC, ': ['Steven Feldman1']},
 {'2Janssen Scientific Affairs, LLC, Horsham PA, ': ['Steve Fakharzadeh2',
   'Jill Abell2',
   'Timothy Hoops2',
   'Bhaskar Srivastava2',
   'Erik Muser2',
   'Danielle Dungee2',
   'Sean Quinn2']},
 {'3HealthiVibe, LLC, Arlington, VA, ': ['Megan Leone Perkins3']},
 {'4University of North Carolina, Chapel Hill, NC, USA ': ['Michael Kappelman4']}]

In [160]:
import itertools

i = list_of_dictionaries[3]
get_key = i.keys()
names = [] 
for key, value in ( 
        itertools.chain.from_iterable( 
            [itertools.product((k, ), v) for k, v in i.items()])): 
                res.append(value)
names

['Michael Kappelman4']

In [161]:
name = [re.sub(r'[0-9]', '', i) for i in names]
print(name)
location = [re.sub(r'[0-9]', '', i) for i in get_key]
print(location)

['Michael Kappelman']
['University of North Carolina, Chapel Hill, NC, USA ']


In [135]:
pattern = re.compile(r"\w+[^A-Z\d\W]\s\w.*")
abstract_title = [re.sub(pattern, "", i) for i in abstract_info]
abstract_title

['GASTROINTESTINAL SYMPTOMS ARE COMMON IN U.S. PATIENTS WITH MODERATESEVERE PSORIASIS ']

In [210]:
abstract = re.findall(r"(Background.*)", get_abstract)
abstract_text = abstract[0]
#abstract_text

In [162]:
import pandas as pd
df2 = pd.DataFrame({"About the person": name[0],
                   "Unnamed: 1": location,
                   "Unnamed: 2": "",
                   "About the session/topic": "P112",
                   "Unnamed: 4": abstract_title,
                   "Unnamed: 5": abstract_text})
df2

,About the person,Unnamed: 1,Unnamed: 2,About the session/topic,Unnamed: 4,Unnamed: 5
0,Michael Kappelman,"University of North Carolina, Chapel Hill, NC,...",,P112,GASTROINTESTINAL SYMPTOMS ARE COMMON IN U.S. P...,Background/Objective:Patients with moderate-to...


In [28]:
df = pd.read_excel('/home/azashiro/Desktop/sheets1.xlsx')

In [208]:
df36 = df35.append(df1, ignore_index=True)
df36

,About the person,Unnamed: 1,Unnamed: 2,About the session/topic,Unnamed: 4,Unnamed: 5
0,Name (incl. titles if any mentioned),Affiliation(s) Name(s),Person's Location,Session Name,Topic Title,Presentation Abstract
1,Zhu Shen,"Institute of Dermatology and Venereology, Sich...",NaN,P100,IMMUNOLOGICAL MEMORY EXISTS IN THE RECURRENT L...,Introduction: Psoriasis usually recurs in prev...
2,Touraj Khosravi-Hafshejani,"Department of Undergraduate Medical Program, F...",NaN,P102,THE IMMUNEPHENOTYPE OF SMALL PLAQUE PSORIASIS,Introduction: Small plaque psoriasis (SPP) is ...
3,Jan Dutz,"Department of Undergraduate Medical Program, F...",NaN,P102,THE IMMUNEPHENOTYPE OF SMALL PLAQUE PSORIASIS,Introduction: Small plaque psoriasis (SPP) is ...
4,Mehran Ghoreishi,"Department of Dermatology, and Skin Science and",NaN,P102,THE IMMUNEPHENOTYPE OF SMALL PLAQUE PSORIASIS,Introduction: Small plaque psoriasis (SPP) is ...
5,Magdalena Martinka,Department of Pahology and Laboratory Medicine...,NaN,P102,THE IMMUNEPHENOTYPE OF SMALL PLAQUE PSORIASIS,Introduction: Small plaque psoriasis (SPP) is ...
6,Cristian Vera Kellet,"Deparment of Dermatology, Pontifical Catholic ...",NaN,P102,THE IMMUNEPHENOTYPE OF SMALL PLAQUE PSORIASIS,Introduction: Small plaque psoriasis (SPP) is ...
7,Goksal Keskin,"Department of Immunology, Medical School of An...",NaN,P104,SERUM GLUCOCORTICOIDINDUCIBLE KINASE 1(SGK1) L...,Introduction: Psoriatic arthritis (PsA) is an ...
8,Umit Olmez,"Department of Immunology, Medical School of An...",NaN,P104,SERUM GLUCOCORTICOIDINDUCIBLE KINASE 1(SGK1) L...,Introduction: Psoriatic arthritis (PsA) is an ...
9,Akiko Arakawa,"Ludwig-Maximilian-University,",NaN,P105,DIFFERENTIAL NH2TERMINAL AUTOANTIGEN TRIMMING ...,Introduction: NH2-terminal trimming by endopla...


In [211]:
'''Import pandas DataFrame into Excel file'''

excel_file = df36.to_excel("/home/azashiro/Desktop/beetroot_task/Data_1.xlsx", index=False)
excel_file